### Modeling (LLM)

#### Setup
(Only general setup, model specific imports are done within sections for the models)

In [1]:
# General (modify where necessary)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### Read data

In [16]:
df = pd.read_csv('../../data/cleaned/Apple_Inc_text_data.csv')

# format datetime again
df['pub_date'] = pd.to_datetime(df['pub_date']) 
df['pub_date'] = df['pub_date'].dt.date

In [17]:
# Copy of df for sentiment analysis, may not actually be necessary, copied from previous nb
sentiment_df=df.copy() #save the df first 
df.head()

,pub_date,abstract,lead_para,headline,doc_type,section_name,type_of_material,rank,web_url
0,2015-04-07,"Want to work at Amazon, Apple or McKinsey? Som...","With some 13,000 graduate schools of business ...",M.B.A. Programs That Get You Where You Want to Go,article,Education,News,7,https://www.nytimes.com/2015/04/12/education/e...
1,2015-04-14,Get recommendations from New York Times report...,Get recommendations from New York Times report...,What We’re Reading,article,Blogs,News,13,https://news.blogs.nytimes.com/2015/04/14/what...
2,2015-04-13,The business unit will partner with companies ...,IBM is taking its Watson artificial-intelligen...,IBM Creates Watson Health to Analyze Medical Data,article,Technology,News,8,https://bits.blogs.nytimes.com/2015/04/13/ibm-...
3,2015-04-22,"With superstars first in line, Apple appears t...","Two weeks ago, Pharrell Williams posted an Ins...",What’s That on Beyoncé’s Wrist? Let Me Guess ....,article,Style,News,1,https://www.nytimes.com/2015/04/23/style/whats...
4,2015-04-01,"In an industry that avoids controversy, the he...",The technology industry’s leaders have found t...,Daily Report: Tech Leaders Come Together to Op...,article,Technology,News,3,https://bits.blogs.nytimes.com/2015/04/01/dail...


#### LLM: Gemini
nerfed by api limit

In [ ]:
# !pip install -U -q "google-genai"

In [ ]:
# Necessary packages
import os
from google import genai
# client = genai.Client(api_key='')

In [19]:
test = sentiment_df[['headline']].tail(10)
test.head()


,headline
2114,"The Tech That Needs Fixing in 2024, and What G..."
2115,"The True Price of Apple’s $3,500 Vision Pro Is..."
2116,Why Making Face Computers Cool Isn’t Easy
2117,How to Cut Down Your Screen Time but Still Get...
2118,Apple to Offer Rare Discount on iPhones in China


In [20]:
import google.genai as genai
import pandas as pd
import time
# Initialize the client
# client = genai.Client()

# Function to get sentiment based on model output
def gemini_predict(prompt):
    # Generate content from the model
    response = client.models.generate_content(
        model='gemini-2.0-flash', 
        contents=prompt
    )
    
    # Assuming the model output is a sentiment label (e.g., Positive, Negative)
    # You may need to adjust based on the actual format of the response
    sentiment = response.text.strip()
    
    # Return the sentiment label, default to 'Neutral' if the model is uncertain
    if sentiment not in ['Positive', 'Negative']:
        sentiment = 'Neutral'
    
    return sentiment

# Function to apply sentiment classification using the prompt for headlines
def find_sentiment_zero_shot(text):
    prompt = f"""Evaluate the sentiment conveyed by the headline with respect to Apple from an investment perspective. 
    Assign one of the following sentiment labels:
    Positive: For headlines with positive implications.
    Negative: For headlines with negative implications.
    Neutral: For headlines with unclear or neutral implications.
    
    Headline: {text}
    Sentiment: """
    
    # Get sentiment from the model
    sentiment = gemini_predict(prompt)
    return sentiment

def apply_with_delay(df, sentiment_column, delay=4):
    sentiment_list = []
    
    for headline in df[sentiment_column]:
        # Apply sentiment classification
        sentiment = find_sentiment_zero_shot(headline)
        sentiment_list.append(sentiment)
        
        # Delay to respect the RPM limit
        time.sleep(delay)  # Delay in seconds (delay = 4 seconds to stay within 15 requests per minute)
    
    # Add the results to the dataframe
    df['gemini_sentiment'] = sentiment_list
    return df

# Apply the function with delay to the sentiment DataFrame
test_df = apply_with_delay(test, 'headline')

# Display the results
print(test_df[['headline', 'gemini_sentiment']])




                                               headline gemini_sentiment
2114  The Tech That Needs Fixing in 2024, and What G...          Neutral
2115  The True Price of Apple’s $3,500 Vision Pro Is...         Negative
2116          Why Making Face Computers Cool Isn’t Easy         Negative
2117  How to Cut Down Your Screen Time but Still Get...          Neutral
2118   Apple to Offer Rare Discount on iPhones in China          Neutral
2119  Apple Takes a Humble Approach to Launching Its...          Neutral
2120  Apple Overhauls App Store in Europe, in Respon...          Neutral
2121  The Apple Vision Pro Is a Marvel. But Who Will...          Neutral
2122  U.S. Moves Closer to Filing Sweeping Antitrust...         Negative
2123                  Charms Can Personalize Your Watch          Neutral


In [21]:
sentiment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2124 entries, 0 to 2123
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   pub_date          2124 non-null   object
 1   abstract          2124 non-null   object
 2   lead_para         2124 non-null   object
 3   headline          2124 non-null   object
 4   doc_type          2124 non-null   object
 5   section_name      2124 non-null   object
 6   type_of_material  2124 non-null   object
 7   rank              2124 non-null   int64 
 8   web_url           2124 non-null   object
dtypes: int64(1), object(8)
memory usage: 149.5+ KB


#### LLM: Llama

In [ ]:
#import dependencies
# !pip install transformers torch peft bitsandbytes accelerate sentencepiece blobfile




  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached peft-0.14.0-py3-none-any.whl.metadata (13 kB)
  Using cached bitsandbytes-0.45.3-py3-none-win_amd64.whl.metadata (5.1 kB)
  Using cached huggingface_hub-0.29.2-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached accelerate-1.4.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.49.0-py3-none-any.whl (10.0 MB)
Using cached peft-0.14.0-py3-none-any.whl (374 kB)
Using cached bitsandbytes-0.45.3-py3-none-win_amd64.whl (75.4 MB)
Using cached accelerate-1.4.0-py3-none-any.whl (342 kB)
Using cached huggingface_hub-0.29.2-py3-none-any.whl (468 kB)
Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl (308 kB)
Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl (2.4 MB)


In [5]:
from transformers import LlamaForCausalLM, LlamaTokenizerFast, BitsAndBytesConfig
from peft import PeftModel  # 0.5.0
import torch

# Load Models
base_model = "NousResearch/Llama-2-13b-hf" 
peft_model = "FinGPT/fingpt-sentiment_llama2-13b_lora"
tokenizer = LlamaTokenizerFast.from_pretrained(base_model)

# Set padding token if needed
tokenizer.pad_token = tokenizer.eos_token

# Define the quantization configuration for 8-bit or 4-bit quantization
quantization_config = BitsAndBytesConfig(load_in_4bit=True)  # You can also use load_in_4bit=True for 4-bit

# Load the model for causal language generation and the LoRA fine-tuned model
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load the model using BitsAndBytesConfig for quantization
model = LlamaForCausalLM.from_pretrained(
    base_model, 
    device_map="auto",  # Automatically distribute layers across available devices
    quantization_config=quantization_config,  # Use quantization configuration
)

model = PeftModel.from_pretrained(model, peft_model)
model.eval()  # Set model to evaluation mode

# Make prompts
prompt = [
    '''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
    Input: FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs.
    Answer: ''',
    '''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
    Input: According to Gran, the company has no plans to move all production to Russia, although that is where the company is growing.
    Answer: ''',
    '''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
    Input: A tinyurl link takes users to a scamming site promising that users can earn thousands of dollars by becoming a Google (NASDAQ: GOOG) Cash advertiser.
    Answer: ''',
]

# Tokenize the prompts
tokens = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Move tensors to the correct device (GPU or CPU)
tokens = {key: value.to(device) for key, value in tokens.items()}

# Generate responses from the model
with torch.no_grad():  # Disable gradients during inference
    res = model.generate(**tokens, max_length=512)

# Decode the results and extract the sentiment part
res_sentences = [tokenizer.decode(i, skip_special_tokens=True) for i in res]

# Extract the answer part (everything after 'Answer: ')
out_text = [o.split("Answer: ")[1].strip() for o in res_sentences]

# Show results
for sentiment in out_text:
    print(sentiment)






c:\Users\Tammy\anaconda3\Lib\site-packages\accelerate\utils\modeling.py:1536: UserWarning: Current model requires 32.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

#### LLM: DeepSeek

In [ ]:
# TO DO